# Building a Diffusion Language Model

A notebook on DLM implementation based on Andrej Karpathy's GPT tutorial

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))
# let's look at the first 1000 characters
print(text[:1000])

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hung

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# for our DLM, we need the [MASK] token
chars = chars + ['[MASK]']
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz[MASK]
66


In [15]:
mask_id = vocab_size - 1
chars[mask_id]

'[MASK]'

In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

## Forward Diffusion

First difference between auto-regressive LMs and DLMs

In [36]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
T = 1000
beta = torch.linspace(1e-4, 0.02, T)   # shape (T,)
alpha = 1.0 - beta
alpha_bar = torch.cumprod(alpha, dim=0)   # shape (T,)

# this is where we need to diverge from Karpathy's code
# input and output is no longer 'text sequence' -> 'next token'
# now it's: 'corrupted block of text' -> 'clean block of text'

def compute_m_t(t):
    """
    t: LongTensor of shape (B,), values in [1, T]
    returns: Tensor of shape (B,), cumulative mask probability
    """
    return 1.0 - alpha_bar[t - 1]


def get_batch(split, device='cpu'):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # clean block of text
    x = torch.stack([data[i:i+block_size] for i in ix])             # (B, L)

    # sample diffusion timestep
    t = torch.randint(1, T + 1, (batch_size,))
    # compute cumulative mask probability m_t
    m_t = compute_m_t(t)  # shape: (B,)
    # sample mask
    mask = torch.rand_like(x.float()) < m_t[:, None]

    # apply absorbing corruption
    xt = x.clone()
    xt[mask] = mask_id

    return xt.to(device), x.to(device), t.to(device)

xt, x, t = get_batch('train')
print('corrupted text:')
print(xt.shape)
print(xt)
print('clean text:')
print(x.shape)
print(x)
print('diffusion timesteps:', t)

corrupted text:
torch.Size([4, 8])
tensor([[65, 65, 58, 65, 65,  1, 46, 65],
        [65, 65, 65, 65, 65, 65, 65, 65],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
clean text:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
diffusion timesteps: tensor([296, 754,  16,  11])


## Reverse Diffusion

A diffusion model's job is actually doing the reverse diffusion (denoising corrupted text to clean, meaningful text in training; or converting a blocked of masked text into readable text in inference)

In [30]:
import torch.nn as nn
from torch.nn import functional as F

class LayerNorm1d: # copied from Karpathy's code

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

In [41]:
# a DLM must use different loss function from an AR model
def diffusion_loss(logits, x0, xt):
    # logits: (B, L, V)
    # x0:     (B, L)
    # xt:     (B, L)

    B, L, V = logits.shape
    logits = logits.view(B * L, V)
    x0 = x0.view(B * L)
    xt = xt.view(B * L)

    mask = (xt == mask_id)
    return F.cross_entropy(logits[mask], x0[mask])

@torch.no_grad()
def estimate_loss(device='cpu'):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xt, x0, t = get_batch(split, device)
            logits = model(xt, t)                 # (B, L, V)
            loss = diffusion_loss(logits, x0, xt)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [32]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)

        # this operation is for building causal mask, which we no longer need
        # self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)

        # we don't use causal mask here, everything else stays the same
        # wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)

        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [33]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [34]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [37]:
# super simple diffusion language model
class DiffusionLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # learned timestep for diffusion
        self.time_embedding = nn.Embedding(T + 1, n_embd)

        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, xt, t=None):
        B, L = xt.shape

        # xt: (B,T) tensor of integers
        tok_emb = self.token_embedding_table(xt) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(L, device=device)) # (T,C)

        time_emb = self.time_embedding(t).unsqueeze(1) #(B,1,C)

        x = tok_emb + pos_emb + time_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        return logits

In [ ]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
T = 100
# ------------
model = DiffusionLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(device=device)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
            # sample a batch of data
    xt, x0, t = get_batch('train', device=device)
    # evaluate the loss
    logits = model(xt, t)
    loss = diffusion_loss(logits, x0, xt)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

0.216322 M parameters
step 0: train loss 4.3858, val loss 4.3668
step 100: train loss 3.3711, val loss 3.3805
step 200: train loss 3.2985, val loss 3.3499
step 300: train loss 3.3654, val loss 3.4109
step 400: train loss 3.3348, val loss 3.3493
step 500: train loss 3.3460, val loss 3.4095
step 600: train loss 3.3052, val loss 3.3649
step 700: train loss 3.3246, val loss 3.3806
step 800: train loss 3.3171, val loss 3.3822
step 900: train loss 3.3316, val loss 3.3767
step 1000: train loss 3.3151, val loss 3.3367
step 1100: train loss 3.2835, val loss 3.3810
step 1200: train loss 3.3372, val loss 3.3491
step 1300: train loss 3.3168, val loss 3.3417
step 1400: train loss 3.3410, val loss 3.4028
step 1500: train loss 3.2802, val loss 3.2910
step 1600: train loss 3.2836, val loss 3.3010
step 1700: train loss 3.2537, val loss 3.2868
step 1800: train loss 3.3169, val loss 3.3216
step 1900: train loss 3.2412, val loss 3.2969
step 2000: train loss 3.3040, val loss 3.3106
step 2100: train loss 3.

In [43]:
@torch.no_grad()
def sample(model, batch_size=1):
    model.eval()
    x = torch.full(
        (batch_size, block_size),
        fill_value=mask_id,
        device=device,
        dtype=torch.long,
    )

    for t in reversed(range(1, T + 1)):
        t_tensor = torch.full((batch_size,), t, device=device, dtype=torch.long)
        logits = model(x, t_tensor)
        probs = torch.softmax(logits, dim=-1)

        # simple unmasking rule (baseline)
        pred = torch.multinomial(
            probs.view(-1, probs.size(-1)),
            num_samples=1,
        ).view(batch_size, block_size)

        mask = (x == mask_id)
        x[mask] = pred[mask]

    return x

samples = sample(m, batch_size=1)
print(decode(samples[0].tolist()))

s m ruoielFs hRyu
u yTon yMtyoeo
